In [1]:
import torch
torch.__version__

'1.13.0.dev20220706+cu113'

##### Usual advice to maximize throughput is to crank your batch size up to you OOM, then back off just a touch...now you have maximized throughput.

##### This is not optimal - you need to back your batch size down both below OOM, and below where you are hitting any cudaMalloc retries! 

In [ ]:
# let's take a 2B ViT model - running FSDP Zero2 sharding, and batch size = 4 to start:

# 5.55 seconds, 4 images 
# 69% of GPU memory is being used

In [ ]:
# let's go until we hit OOM:
# BS= 24

In [3]:
# let's back off until we don't get OOM:
# bs=17


In [2]:
# 'by hand' way to monitor gpu memory to avoid cudaMalloc retries...
torch.cuda.memory_summary()
            

18907136.0

In [ ]:
# by code way
cuda_info = torch.cuda.memory_stats()

num_retries = cuda_info.get("num_alloc_retries", 0)
cuda_ooms = cuda_info.get("num_ooms", 0)

print(f"cudaMalloc retries = {num_retries}")
print(f"cuda OOM = {cuda_ooms}\n")

In [4]:
# with utility class Memory_Maximizer:
# setup memory tracking for perf
from gpu_memory import Memory_Maximizer

# setup memory tracking for perf
    if rank == 0:
        memmax = Memory_Maximizer()

# memory and timing tracking
    if local_rank == 0:
        memmax.start()  # start will reset all tracking points

# in training loop - at minibatch or epoch end point:
    loss.backward()
    optimizer.step()

    # update durations and memory tracking
    if local_rank == 0:
        memmax.update()

# at end of training - stop and print stats
    # memory summary
    if local_rank == 0:
        memmax.stop()  # stop and display info  

IndentationError: unexpected indent (422830102.py, line 4)